# Step 1: Load and Inspect Data Files

In [222]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile

In [223]:
MOD = 1e18 + 3
fact = [0 for _ in range(1000000)]
fact = np.array(fact)
fact[0] = 1
fact[1] = 1
for i in range(2, len(fact)):
    fact[i] = (fact[i-1] * i) % MOD

/var/folders/kd/99328vjn0zlc1kls78_rwvdh0000gn/T/ipykernel_33178/497385358.py:7: RuntimeWarning: overflow encountered in scalar multiply
  fact[i] = (fact[i-1] * i) % MOD


In [224]:
# Use numba here (the C thingy)
# Use xxhash --> https://stackoverflow.com/questions/9350002/python-tuples-as-keys-slow,
# https://github.com/Cyan4973/xxHash

# @jit(nopython=True, parallel = True, fastmath = True)
def c_hash(perm):
    return tuple(perm)
    # base = 9973
    # modb = 1000000007
    # modc = 1000000009

    # B, C = 0, 0
    # for i in perm:
    #     B = (B * base) % modb + i
    #     C = (C * base) % modc + i

    # return (B, C)


#wegithe dist by size ~ needs to scale with path length (dist increases linearly, path increases exponentially (?) )
# @jit(nopython=True, parallel = True, fastmath = True)
def dist(a, b):
    return np.count_nonzero(a != b)

arrA = [i for i in range(100000000)]
arrB = [i + 2 for i in range(100000000)]

cProfile.run("dist(arrA, arrB)")

         7 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 114246932.py:22(dist)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 numeric.py:411(_count_nonzero_dispatcher)
        1    0.000    0.000    0.000    0.000 numeric.py:415(count_nonzero)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.count_nonzero}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [225]:
def getInversePerm(arr):
    # gets the inverse move for a certain move
    res = [0 for i in range(len(arr))]
    for i in range(len(arr)):
        res[arr[i]] = i
    return res

In [226]:
import pandas as pd

# File paths
puzzle_info_path = 'puzzle_info.csv'
puzzles_path = 'puzzles.csv'
sample_submission_path = 'submission.csv' # change name across board

# Loading the data
puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)

# Displaying the first few rows of each dataframe
puzzles_df.describe()

,id,num_wildcards
count,398.000000,398.000000
mean,198.500000,1.809045
std,115.036951,10.047931
min,0.000000,0.000000
25%,99.250000,0.000000
50%,198.500000,0.000000
75%,297.750000,0.000000
max,397.000000,176.000000


In [227]:
puzzles_df.head()

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0


In [228]:
print(len(puzzles_df))

398


In [229]:
mp = {}
mp_s_num = {}

for i in range(len(puzzles_df)):
    if(puzzles_df['puzzle_type'][i] not in mp):
        mp[puzzles_df['puzzle_type'][i]] = 0
        mp_s_num[puzzles_df['puzzle_type'][i]] = len(puzzles_df['solution_state'][i].split(';'))
    mp[puzzles_df['puzzle_type'][i]] += 1

In [230]:
print(mp)

{'cube_2/2/2': 30, 'cube_3/3/3': 120, 'cube_4/4/4': 60, 'cube_5/5/5': 35, 'cube_6/6/6': 12, 'cube_7/7/7': 5, 'cube_8/8/8': 5, 'cube_9/9/9': 5, 'cube_10/10/10': 5, 'cube_19/19/19': 4, 'cube_33/33/33': 3, 'wreath_6/6': 20, 'wreath_7/7': 15, 'wreath_12/12': 10, 'wreath_21/21': 5, 'wreath_33/33': 3, 'wreath_100/100': 1, 'globe_1/8': 10, 'globe_1/16': 5, 'globe_2/6': 5, 'globe_3/4': 15, 'globe_6/4': 5, 'globe_6/8': 5, 'globe_6/10': 5, 'globe_3/33': 4, 'globe_33/3': 4, 'globe_8/25': 2}


In [231]:
print(mp_s_num)

{'cube_2/2/2': 24, 'cube_3/3/3': 54, 'cube_4/4/4': 96, 'cube_5/5/5': 150, 'cube_6/6/6': 216, 'cube_7/7/7': 294, 'cube_8/8/8': 384, 'cube_9/9/9': 486, 'cube_10/10/10': 600, 'cube_19/19/19': 2166, 'cube_33/33/33': 6534, 'wreath_6/6': 10, 'wreath_7/7': 12, 'wreath_12/12': 22, 'wreath_21/21': 40, 'wreath_33/33': 64, 'wreath_100/100': 198, 'globe_1/8': 32, 'globe_1/16': 64, 'globe_2/6': 36, 'globe_3/4': 32, 'globe_6/4': 56, 'globe_6/8': 112, 'globe_6/10': 140, 'globe_3/33': 264, 'globe_33/3': 264, 'globe_8/25': 450}


In [239]:
# Number of edges

for i in range(len(puzzle_info_df)):
    print(len(puzzle_info_df['allowed_moves'][i].split(':')))

7
10
13
16
19
22
25
28
31
58
100
3
3
3
3
3
3
19
35
16
13
16
24
28
71
71
60


In [198]:
sample_submission_df.head()

,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...


In [199]:
tot = 0
arr = []
for i in range(len(sample_submission_df)):
    arr.append(len(sample_submission_df['moves'][i].split('.')))
    tot += len(sample_submission_df['moves'][i].split('.'))

def describe_array(arr):
    print("Array:", arr)
    print("Count:", len(arr))
    print("Mean:", np.mean(arr))
    print("Median:", np.median(arr))
    print("Standard Deviation:", np.std(arr))
    print("Variance:", np.var(arr))
    print("Minimum:", np.min(arr))
    print("Maximum:", np.max(arr))
    print("Range:", np.max(arr) - np.min(arr))
    print("25th Percentile:", np.percentile(arr, 25))
    print("50th Percentile (Median):", np.percentile(arr, 50))
    print("75th Percentile:", np.percentile(arr, 75))
    print("Interquartile Range:", np.percentile(arr, 75) - np.percentile(arr, 25))


describe_array(arr)


Array: [2, 9, 62, 92, 70, 54, 10, 83, 98, 76, 10, 63, 72, 131, 96, 10, 11, 62, 11, 82, 112, 96, 63, 53, 99, 61, 93, 73, 83, 82, 300, 392, 268, 239, 304, 440, 267, 299, 237, 408, 259, 296, 208, 229, 602, 584, 327, 358, 242, 258, 238, 370, 479, 355, 230, 309, 285, 344, 282, 415, 251, 236, 385, 225, 429, 200, 250, 331, 298, 225, 205, 266, 327, 392, 408, 320, 382, 358, 257, 254, 372, 221, 251, 300, 260, 227, 265, 310, 209, 268, 277, 303, 216, 214, 317, 287, 425, 323, 292, 249, 263, 356, 274, 398, 221, 231, 207, 340, 342, 268, 257, 265, 399, 219, 262, 273, 329, 315, 385, 360, 331, 295, 339, 346, 301, 311, 268, 264, 260, 299, 362, 401, 215, 268, 395, 372, 227, 376, 257, 241, 209, 287, 412, 474, 399, 267, 358, 263, 233, 258, 700, 610, 498, 593, 665, 544, 642, 637, 443, 697, 598, 538, 789, 816, 542, 616, 597, 465, 420, 670, 558, 519, 466, 736, 483, 472, 737, 493, 534, 486, 670, 592, 491, 675, 499, 888, 595, 734, 701, 451, 458, 731, 737, 659, 495, 594, 947, 550, 422, 517, 623, 633, 439, 586, 50

In [200]:
tot = 0
arr = []
for i in range(len(puzzles_df)):
    arr.append(puzzles_df['num_wildcards'][i])
    tot += puzzles_df['num_wildcards'][i]

def describe_array(arr):
    print("Array:", arr)
    print("Count:", len(arr))
    print("Mean:", np.mean(arr))
    print("Median:", np.median(arr))
    print("Standard Deviation:", np.std(arr))
    print("Variance:", np.var(arr))
    print("Minimum:", np.min(arr))
    print("Maximum:", np.max(arr))
    print("Range:", np.max(arr) - np.min(arr))
    print("25th Percentile:", np.percentile(arr, 25))
    print("50th Percentile (Median):", np.percentile(arr, 50))
    print("75th Percentile:", np.percentile(arr, 75))
    print("Interquartile Range:", np.percentile(arr, 75) - np.percentile(arr, 25))


describe_array(arr)


Array: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 6, 0, 0, 0, 0, 6, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0, 2, 0, 0, 4, 0, 0, 0, 4, 0, 2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 6, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 8, 0, 0, 2, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 10, 0, 0, 10, 0, 6, 0, 0, 10, 0, 0, 0, 6, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 2, 0, 0, 10, 2, 0, 2, 2, 18, 0, 0, 0, 0, 0, 0, 16, 42, 0, 0, 0, 0, 0, 0, 34, 18, 0, 38, 0, 0, 16, 0, 0, 0, 0, 0, 54, 176, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0,

In [201]:
print(tot)

720


# Step 2: Parse puzzle status

In [202]:
# Parsing the initial_state and solution_state columns
# Converting the semicolon-separated string values into lists of colors
puzzles_df['parsed_initial_state'] = puzzles_df['initial_state'].apply(lambda x: x.split(';'))
seen = {}

for i in range(len(puzzles_df['parsed_initial_state'])):
    for j in range(len(puzzles_df['parsed_initial_state'][i])):
        if puzzles_df['parsed_initial_state'][i][j] not in seen:
            seen[puzzles_df['parsed_initial_state'][i][j]] = len(seen)
        puzzles_df['parsed_initial_state'][i][j] = seen[puzzles_df['parsed_initial_state'][i][j]]

puzzles_df['parsed_solution_state'] = puzzles_df['solution_state'].apply(lambda x: x.split(';'))

for i in range(len(puzzles_df['parsed_solution_state'])):
    for j in range(len(puzzles_df['parsed_solution_state'][i])):
        puzzles_df['parsed_solution_state'][i][j] = seen[puzzles_df['parsed_solution_state'][i][j]]

# Displaying the modified dataframe with parsed states
puzzles_df[['id', 'puzzle_type', 'parsed_initial_state', 'parsed_solution_state']].head()

,id,puzzle_type,parsed_initial_state,parsed_solution_state
0,0,cube_2/2/2,"[0, 1, 0, 2, 1, 3, 2, 3, 4, 2, 4, 2, 0, 4, 0, ...","[2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 0, 0, 0, ..."
1,1,cube_2/2/2,"[0, 1, 4, 3, 3, 1, 5, 2, 5, 0, 3, 5, 5, 1, 3, ...","[2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 0, 0, 0, ..."
2,2,cube_2/2/2,"[1, 5, 4, 4, 5, 2, 0, 0, 3, 3, 2, 5, 1, 3, 4, ...","[2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 0, 0, 0, ..."
3,3,cube_2/2/2,"[2, 4, 1, 4, 5, 0, 1, 0, 2, 2, 5, 2, 3, 0, 3, ...","[2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 0, 0, 0, ..."
4,4,cube_2/2/2,"[1, 0, 1, 0, 2, 1, 5, 3, 2, 4, 5, 0, 5, 0, 4, ...","[2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 0, 0, 0, ..."


# Step 3: Deal with allow move

In [203]:
import json

# Converting the string representation of allowed_moves to dictionary
puzzle_info_df['allowed_moves'] = puzzle_info_df['allowed_moves'].apply(lambda x: json.loads(x.replace("'", '"')))

# Selecting an example puzzle type and displaying its allowed moves
example_puzzle_type = puzzle_info_df['puzzle_type'].iloc[0]
example_allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == example_puzzle_type]['allowed_moves'].iloc[0]

example_puzzle_type

'cube_2/2/2'

In [204]:
pd.DataFrame(example_allowed_moves)

,f0,f1,r0,r1,d0,d1
0,0,18,0,4,0,1
1,1,16,5,1,1,3
2,19,2,2,6,2,0
3,17,3,7,3,3,2
4,6,4,4,20,4,16
5,4,5,21,5,5,17
6,7,6,6,22,18,6
7,5,7,23,7,19,7
8,2,8,10,8,8,4
9,9,0,8,9,9,5


In [205]:
print(puzzle_info_df.head())

  puzzle_type                                      allowed_moves
0  cube_2/2/2  {'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...
1  cube_3/3/3  {'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ...
2  cube_4/4/4  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
3  cube_5/5/5  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
4  cube_6/6/6  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...


In [206]:
print(puzzle_info_df['allowed_moves'][0])

{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11, 12, 13, 14, 15, 16, 20, 18, 21, 10, 8, 22, 23], 'f1': [18, 16, 2, 3, 4, 5, 6, 7, 8, 0, 10, 1, 13, 15, 12, 14, 22, 17, 23, 19, 20, 21, 11, 9], 'r0': [0, 5, 2, 7, 4, 21, 6, 23, 10, 8, 11, 9, 3, 13, 1, 15, 16, 17, 18, 19, 20, 14, 22, 12], 'r1': [4, 1, 6, 3, 20, 5, 22, 7, 8, 9, 10, 11, 12, 2, 14, 0, 17, 19, 16, 18, 15, 21, 13, 23], 'd0': [0, 1, 2, 3, 4, 5, 18, 19, 8, 9, 6, 7, 12, 13, 10, 11, 16, 17, 14, 15, 22, 20, 23, 21], 'd1': [1, 3, 0, 2, 16, 17, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13, 18, 19, 20, 21, 22, 23]}


In [207]:
# type : (np.array(move_perm_i), np.array(name_i))
puz_info = {}

for i in range(len(puzzle_info_df)):
    puz_info[puzzle_info_df['puzzle_type'][i]] = [[], []]
    for j in puzzle_info_df['allowed_moves'][i].keys():
        puz_info[puzzle_info_df['puzzle_type'][i]][1].append(j)
        puz_info[puzzle_info_df['puzzle_type'][i]][0].append(np.array(puzzle_info_df['allowed_moves'][i][j]))

        puz_info[puzzle_info_df['puzzle_type'][i]][1].append(str('-' + j)) # might be the opposite
        puz_info[puzzle_info_df['puzzle_type'][i]][0].append(np.array(getInversePerm(puzzle_info_df['allowed_moves'][i][j])))

In [208]:
print(puz_info['cube_3/3/3'])

[[array([ 0,  1,  2,  3,  4,  5, 44, 41, 38, 15, 12,  9, 16, 13, 10, 17, 14,
       11,  6, 19, 20,  7, 22, 23,  8, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 45, 39, 40, 46, 42, 43, 47, 24, 21, 18, 48, 49, 50,
       51, 52, 53]), array([ 0,  1,  2,  3,  4,  5, 18, 21, 24, 11, 14, 17, 10, 13, 16,  9, 12,
       15, 47, 19, 20, 46, 22, 23, 45, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37,  8, 39, 40,  7, 42, 43,  6, 38, 41, 44, 48, 49, 50,
       51, 52, 53]), array([ 0,  1,  2, 43, 40, 37,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18,  3, 20, 21,  4, 23, 24,  5, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 48, 38, 39, 49, 41, 42, 50, 44, 45, 46, 47, 25, 22, 19,
       51, 52, 53]), array([ 0,  1,  2, 19, 22, 25,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 50, 20, 21, 49, 23, 24, 48, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36,  5, 38, 39,  4, 41, 42,  3, 44, 45, 46, 47, 37, 40, 43,
       51, 52, 53]), array([42, 39,

# Step 4: Apply actions to the puzzle state

In [209]:
t_n = len(puz_info['cube_2/2/2'][0][0])

a = puz_info['cube_2/2/2'][0][0]
_a = puz_info['cube_2/2/2'][0][1]
arr = np.array(puzzles_df['parsed_initial_state'][0])

print(arr[a])
print(arr[_a])

print("Inverse check:")
print(arr[a][_a])
print(arr[a][_a] == arr)
print(arr[_a][a])

[0 1 1 5 2 1 3 3 0 2 2 2 0 4 0 5 5 3 1 5 4 4 3 4]
[0 1 4 4 3 3 1 2 5 2 3 2 0 4 0 5 5 2 1 0 5 1 3 4]
Inverse check:
[0 1 0 2 1 3 2 3 4 2 4 2 0 4 0 5 5 5 1 1 3 5 3 4]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True]
[0 1 0 2 1 3 2 3 4 2 4 2 0 4 0 5 5 5 1 1 3 5 3 4]


# Step 5: Develop search algorithm

In [210]:
print(sample_submission_df.head())

   id                                              moves
0   0                                             r1.-f1
1   1                     f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2   2  f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3   3  -f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4   4  d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...


In [211]:
@jit(nopython=True, parallel = True, fastmath = True)
def hash(perm):
    base = 9973
    modb = 1000000007
    modc = 1000000009

    B, C = 0, 0
    for i in perm:
        B = (B * base) % modb + i
        C = (C * base) % modc + i

    return (B, C)

In [212]:
mx_mem = int(3e7)
memory = [['', 0, []] for i in range(mx_mem)]
# move_to_get_here, last_idx, perm

In [213]:
# memory = np.array(memory) # call np.zeroes directly
mem_idx = 0

In [216]:
def a_star_search(initial_state, goal_state, max_dist, move_perm, move_name):
    global mem_idx
    
    n = len(initial_state)
    m = len(move_perm)
    
    # Priority queue for the open set (states to be evaluated)
    open_set = []
    heapq.heappush(open_set, (0, mem_idx))  # (priority, state, mem_idx)
    memory[mem_idx][2] = initial_state
    mem_idx += 1

    # all considered permutations
    considered = set()
    considered.add(hash(initial_state))

    while mem_idx + 200 < mx_mem:
        cur_p, cur_idx = heapq.heappop(open_set)
        cur_state = memory[cur_idx][2]
        
        cur_dist = dist(cur_state, goal_state)
        cur_p -= cur_dist 
        # cur_state = np.array(cur_state)

        #Comparision optimize: Just use hash if no wild_card
        if cur_dist <= max_dist:
            return cur_idx

        for i in range(m):
            perm = move_perm[i]
            name = move_name[i]

            new_state = cur_state[perm]
            cur_hash = hash(new_state)
            
            if cur_hash not in considered:
                considered.add(cur_hash)
                memory[mem_idx][0] = name
                memory[mem_idx][1] = cur_idx
                memory[mem_idx][2] = new_state
                
                priority = cur_p + 1 + dist(new_state, goal_state) #might not be doing much. Regular dijkstra?
                
                heapq.heappush(open_set, (priority, mem_idx))
                mem_idx += 1
    return None

In [221]:
def dig_through_memory(idx):
    res = []
    while idx != 0:
        # print(idx)
        res.append(memory[idx][0])
        idx = memory[idx][1]

    res = list(reversed(res))
    return res

def solve(idx):
    global mem_idx
    
    mem_idx = 0
    
    initial_state = np.array(puzzles_df['parsed_initial_state'][idx])
    goal_state = np.array(puzzles_df['parsed_solution_state'][idx])
    max_dist = puzzles_df['num_wildcards'][idx]
    
    move_perm = np.array(puz_info[puzzles_df['puzzle_type'][idx]][0])
    move_name = np.array(puz_info[puzzles_df['puzzle_type'][idx]][1])

    res = a_star_search(initial_state, goal_state, max_dist, move_perm, move_name)
    
    if res != None:
        return dig_through_memory(res)
        
    return sample_submission_df['moves'][idx]

cProfile.run("solve(2)")
# solve(0)

         6012303 function calls in 3.125 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   856125    0.505    0.000    0.708    0.000 114246932.py:22(dist)
        1    1.671    1.671    2.996    2.996 1586048460.py:1(a_star_search)
        1    0.000    0.000    0.000    0.000 2844175703.py:1(dig_through_memory)
        1    0.127    0.127    3.125    3.125 2844175703.py:11(solve)
   954925    0.304    0.000    0.304    0.000 327325576.py:1(hash)
        1    0.000    0.000    3.125    3.125 <string>:1(<module>)
        5    0.000    0.000    0.000    0.000 __init__.py:33(using_copy_on_write)
        5    0.000    0.000    0.000    0.000 base.py:5299(__contains__)
       10    0.000    0.000    0.000    0.000 common.py:367(apply_if_callable)
        5    0.000    0.000    0.000    0.000 frame.py:3856(__getitem__)
        5    0.000    0.000    0.000    0.000 frame.py:4405(_get_item_cache)
       10    0.000    0.000    0.

# Step 6: Build submission format function

In [30]:
def format_solution_for_submission(puzzle_id, solution_moves):
    """
    Format the solution to a puzzle for submission.

    :param puzzle_id: The unique identifier of the puzzle.
    :param solution_moves: List of tuples representing the solution moves.
    :return: Formatted string suitable for submission.
    """
    formatted_moves = []
    for move in solution_moves:
        formatted_moves.append(move)

    # Joining the moves into a single string separated by periods
    return {'id': puzzle_id, 'moves': '.'.join(formatted_moves)}


# Step 7: Define solve function

In [31]:
from tqdm import tqdm

res_df = []

for i in tqdm(range(len(puzzles_df))):
    res = solve(i)
    # print(res)
    if res == None:
        res = format_solution_for_submission(i, sample_submission_df['moves'][i].split('.'))
    else:
        res = format_solution_for_submission(i, res)
    res_df.append(res)

 12%|████▍                               | 49/398 [8:50:41<62:59:48, 649.83s/it]


KeyboardInterrupt: 

In [ ]:
res_df = pd.DataFrame(res_df)
res_df

# Step 9: Save submission

In [ ]:
# Define the file path for the output CSV file
output_csv_path = 'submission.csv'

# Save the output DataFrame to a CSV file
res_df.to_csv(output_csv_path, index=False)

# Return the path of the saved file
output_csv_path